In [ ]:
import os
import csv
import pandas as pd

In [ ]:
hmda = pd.read_csv('../input_data/state_AZ-CA_mini.csv')

In [ ]:
hmda['denial'] = hmda['action_taken'] == 3

In [ ]:
def hmda_bank(row):
  if row['lei'] == 'QOT5WN9RBKQTFRVKEV31':
    hmda_bank = 'Bank of West'
  elif row['lei'] == '3Y4U8VZURTYWI1W2K376':
    hmda_bank = 'BMO Harris'
  else: 
    hmda_bank = 'All Other Banks'
  return hmda_bank

In [ ]:
hmda['which_bank'] = hmda.apply(hmda_bank,axis=1)
hmda['which_bank'].value_counts()

In [ ]:
hmda.groupby(['which_bank', 'loan_type', 'loan_purpose'])['denial'].mean()

In [ ]:
# Create the pivot table
pivot = hmda.pivot_table(index=['loan_type', 'loan_purpose'], columns = ['which_bank'], values='denial', aggfunc='mean')

# The pivot table shows the mean denial rate for each combination of bank, loan type, and loan purpose
print(pivot)


In [ ]:
pivot['diff_BMO'] = pivot['BMO Harris'] - pivot['All Other Banks']
pivot['diff_BOW'] = pivot['Bank of West'] - pivot['All Other Banks']
pivot.style.format('{:,.2f}')
print(pivot)